In [3]:
# I import the necessary libraries to manage environment variables, interact with OpenAI, and connect to Milvus
import os
from dotenv import load_dotenv
from openai import OpenAI
from pymilvus import Collection, CollectionSchema, FieldSchema, DataType

# I load the environment variables from my .env file where my API key is stored securely
load_dotenv()

# I initialize the OpenAI client using my API key so I can generate embeddings and use OpenAI services
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# From here, I’m ready to create embeddings with OpenAI and perform vector searches using Milvus

In [7]:
from pymilvus import connections

# I establish a connection to the Milvus server.
# Replace "localhost" and 19530 with your Milvus server's actual host and port if different.
connections.connect(
    alias="default",       # I give this connection an alias to reference it later
    host="localhost",      # The host where Milvus is running
    port="19530"           # The port Milvus listens on (default is 19530)
)

print("Connection to Milvus established.")

Connection to Milvus established.


In [11]:
from pymilvus import Collection, list_collections

collection_name = "appliance_reviews"

# Check if the collection already exists in Milvus
if collection_name in list_collections():
    # If it exists, drop it to avoid schema conflicts
    existing_collection = Collection(collection_name)
    existing_collection.drop()
    print(f"Dropped existing collection '{collection_name}'.")

# Define your schema fields here (replace with your actual schema)
from pymilvus import FieldSchema, CollectionSchema, DataType

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1536),
    FieldSchema(name="combined_text", dtype=DataType.VARCHAR, max_length=1024),
    FieldSchema(name="unixReviewTime", dtype=DataType.INT64)
]

schema = CollectionSchema(fields=fields, description="Amazon appliance reviews with vectors and timestamps")

# Create the collection with the schema
collection = Collection(name=collection_name, schema=schema)
print(f"Created collection '{collection_name}' with the specified schema.")

Dropped existing collection 'appliance_reviews'.
Created collection 'appliance_reviews' with the specified schema.


In [17]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 17.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 20.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2


In [5]:
import pandas as pd

# Load JSON files into DataFrames
metadata_path = '/Users/daniel/Documents/Northwestern/MSDS 420 - Database Systems/Final Project Implementation/Final_Project_Phase_1/Amazon/Amazon_Appliances_Metadata.json'
reviews_path = '/Users/daniel/Documents/Northwestern/MSDS 420 - Database Systems/Final Project Implementation/Final_Project_Phase_1/Amazon/Amazon_Appliances_Reviews.json'

metadata_df = pd.read_json(metadata_path, lines=True)
reviews_df = pd.read_json(reviews_path, lines=True)

# Print shape and columns for both DataFrames
print(f"Metadata shape: {metadata_df.shape}")
print(f"Metadata columns: {metadata_df.columns.tolist()}")
print(f"Reviews shape: {reviews_df.shape}")
print(f"Reviews columns: {reviews_df.columns.tolist()}")

Metadata shape: (11656, 9)
Metadata columns: ['asin', 'price', 'imUrl', 'description', 'categories', 'title', 'brand', 'related', 'salesRank']
Reviews shape: (143685, 9)
Reviews columns: ['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime']


In [7]:
# Merge metadata and reviews on 'asin' to get product info alongside reviews
merged_df = reviews_df.merge(metadata_df[['asin', 'title', 'price']], on='asin', how='left')

# Create a combined text column from 'summary' and 'reviewText'
merged_df['combined_text'] = (merged_df['summary'].fillna('') + ' ' + merged_df['reviewText'].fillna('')).str.strip()

# Check the first 3 rows to verify
print(merged_df[['asin', 'title', 'price', 'combined_text']].head(3))

         asin                                              title  price  \
0  0970408285                                                NaN  40.75   
1  7301113188                                                NaN    NaN   
2  B00002N7HY  Leviton 5050 50 Amp, 125/250 Volt, NEMA 10-50R...   2.29   

                                       combined_text  
0  Good fit Could have been longer though. well m...  
1  I Love the Freezer storage line.. I like these...  
2  expectations achieved. It works, no fires, etc...  


In [11]:
from sentence_transformers import SentenceTransformer

# Load the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for the first 100 rows combined_text
sample_texts = merged_df['combined_text'].head(100).tolist()
embeddings = model.encode(sample_texts)

print(f"Generated embeddings for {len(embeddings)} samples.")

Generated embeddings for 100 samples.


In [13]:
# Truncate combined_text to 1024 characters (Milvus VARCHAR limit)
texts_to_insert = merged_df['combined_text'].head(100).apply(lambda x: x[:1024]).tolist()

# Extract unixReviewTime for the first 100 rows
timestamps_to_insert = merged_df['unixReviewTime'].head(100).tolist()

# embeddings already generated, make sure to convert to list if not already
embeddings_to_insert = embeddings.tolist()

print(f"Prepared {len(embeddings_to_insert)} embeddings, {len(texts_to_insert)} texts, and {len(timestamps_to_insert)} timestamps for insertion.")

Prepared 100 embeddings, 100 texts, and 100 timestamps for insertion.


In [19]:
from pymilvus import connections

# Connect to Milvus server
connections.connect(
    alias="default",  # connection alias
    host="localhost", # or your Milvus server host
    port="19530"      # default Milvus port
)

print("Connected to Milvus server.")

Connected to Milvus server.


In [25]:
from pymilvus import connections, Collection, CollectionSchema, FieldSchema, DataType, list_collections

# Connect first
connections.connect(alias="default", host="localhost", port="19530")

collection_name = "appliance_reviews"

# Drop collection if exists
if collection_name in list_collections():
    Collection(collection_name).drop()

# Define schema with dim=384
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
    FieldSchema(name="combined_text", dtype=DataType.VARCHAR, max_length=1024),
    FieldSchema(name="unixReviewTime", dtype=DataType.INT64),
]

schema = CollectionSchema(fields, description="Amazon appliance reviews with 384-dim vectors")

# Create collection
collection = Collection(name=collection_name, schema=schema)

print(f"Created collection '{collection_name}' with 384-dimensional vectors.")

Created collection 'appliance_reviews' with 384-dimensional vectors.


In [27]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Assuming merged_df is already loaded and has 'combined_text' and 'unixReviewTime'

# Load the sentence transformer model (outputs 384-dim embeddings)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Select the first 100 samples (you can increase later)
sample_df = merged_df.head(100)

# Generate embeddings for combined_text column
embeddings = model.encode(sample_df['combined_text'].tolist())

# Truncate combined_text to 1024 characters for Milvus varchar field
texts_to_insert = sample_df['combined_text'].apply(lambda x: x[:1024]).tolist()

# Extract unixReviewTime as list
timestamps_to_insert = sample_df['unixReviewTime'].tolist()

# Convert embeddings to list of lists if not already
embeddings_to_insert = embeddings.tolist()

print(f"Prepared {len(embeddings_to_insert)} embeddings, {len(texts_to_insert)} texts, and {len(timestamps_to_insert)} timestamps for insertion.")

Prepared 100 embeddings, 100 texts, and 100 timestamps for insertion.


In [29]:
from pymilvus import connections, Collection

# Connect to Milvus server (if not already connected)
connections.connect(
    alias="default",
    host="localhost",
    port="19530"
)

# Load the collection (make sure schema dimension is 384)
collection = Collection("appliance_reviews")

# Insert the data - order must match schema fields: embedding, combined_text, unixReviewTime
collection.insert([
    embeddings_to_insert,
    texts_to_insert,
    timestamps_to_insert
])

print(f"Inserted {len(texts_to_insert)} records into Milvus.")

Inserted 100 records into Milvus.


In [31]:
# Requirement 1:
# Create a table of queries for the selected group (Group C in this example)
# including 3 custom queries I created in Phase 1 that fit the same group.

import pandas as pd

# Queries from Group C (Hybrid Search Queries) as per Final Project doc
group_c_queries = [
    "Get a list of those reviews that are similar to this text: 'It could so nearly have been a great fridge, but it's the design that brings it down. The doors are clunky, and the buttons on the outside are unappealing and cheap looking.'",
    "Get a list of those reviews that are similar to this text and got reviewed in February of 2013.",
    "Get a list of those reviews that are similar to this text: 'This microwave started to make a lot of noise after using it for only few days' and got reviewed in February of 2013."
]

# My 3 custom queries from Phase 1 for Group C
custom_queries = [
    "Find reviews semantically similar to: 'Looks great but doesn’t last long.'",
    "Retrieve reviews similar to: 'Powerful motor and easy to clean' for products under the 'Blender' category.",
    "Get reviews expressing frustration with delayed shipping or broken packaging, even if those exact words aren’t used."
]

# Combine all queries into a DataFrame
queries_df = pd.DataFrame({
    "Query": group_c_queries + custom_queries
})

print("Requirement 1 - Combined Queries for Group C:")
print(queries_df)

Requirement 1 - Combined Queries for Group C:
                                               Query
0  Get a list of those reviews that are similar t...
1  Get a list of those reviews that are similar t...
2  Get a list of those reviews that are similar t...
3  Find reviews semantically similar to: 'Looks g...
4  Retrieve reviews similar to: 'Powerful motor a...
5  Get reviews expressing frustration with delaye...


In [33]:
# Requirement 2 - Database Selection and Benchmarking Metrics

# For the queries listed in Requirement 1 (Group C - Hybrid Search Queries),
# I have selected Milvus as the database system because it excels at handling
# similarity and semantic searches with vector embeddings.

# Benchmarking Metrics for Milvus (Vector Database):
# 1. Similarity Search Performance - How quickly and accurately the database
#    finds vectors close to a query vector.
# 2. Indexing Efficiency - Speed and resource usage to create and maintain
#    vector indexes.
# 3. Storage Efficiency - How well the database compresses and stores high-
#    dimensional vector data for scalability.

selected_database = "Milvus"

benchmark_metrics = [
    "Similarity Search Performance",
    "Indexing Efficiency",
    "Storage Efficiency"
]

print("Selected Database:", selected_database)
print("Benchmarking Metrics:")
for metric in benchmark_metrics:
    print(f"- {metric}")

Selected Database: Milvus
Benchmarking Metrics:
- Similarity Search Performance
- Indexing Efficiency
- Storage Efficiency


In [47]:
# Requirement 3 - Design and develop a Milvus database application
# I connected to Milvus, created a collection schema; if it doesn't exist, I'll create it,
# I prepared the data by merging reviews and metadata, generate embeddings,
# insert the data into Milvus, create an index on the vector field,
# and load the collection for queries.

from pymilvus import connections, Collection, list_collections, FieldSchema, CollectionSchema, DataType
import pandas as pd
from sentence_transformers import SentenceTransformer

# Step 1: Connect to the Milvus server
connections.connect(alias="default", host="localhost", port="19530")
print("Connected to Milvus")

# Step 2: Define the collection schema with vector dimension 384 (matching embeddings)
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
    FieldSchema(name="combined_text", dtype=DataType.VARCHAR, max_length=1024),
    FieldSchema(name="unixReviewTime", dtype=DataType.INT64),
]
schema = CollectionSchema(fields, description="Amazon appliance reviews with embeddings")

# Step 3: Create or load the collection
collection_name = "appliance_reviews"
if collection_name in list_collections():
    collection = Collection(collection_name)
    print(f"Loaded existing collection '{collection_name}'")
else:
    collection = Collection(name=collection_name, schema=schema)
    print(f"Created new collection '{collection_name}'")

# Step 4: Prepare the data
# Ensure metadata_df and reviews_df are already loaded
merged_df = reviews_df.merge(metadata_df[['asin', 'title', 'price']], on='asin', how='left')
merged_df['combined_text'] = (merged_df['summary'].fillna('') + ' ' + merged_df['reviewText'].fillna('')).str.strip()

# Step 5: Load embedding model and generate embeddings for first 100 rows
model = SentenceTransformer('all-MiniLM-L6-v2')
sample_df = merged_df.head(100).copy()
sample_df['embedding'] = sample_df['combined_text'].apply(lambda x: model.encode(x).tolist())

# Step 6: Prepare lists for insertion (truncate text and extract timestamps)
texts_to_insert = sample_df['combined_text'].apply(lambda x: x[:1024]).tolist()
embeddings_to_insert = sample_df['embedding'].tolist()
timestamps_to_insert = sample_df['unixReviewTime'].tolist()

# Step 7: Insert the data into the Milvus collection
collection.insert([embeddings_to_insert, texts_to_insert, timestamps_to_insert])
print(f"Inserted {len(texts_to_insert)} records into Milvus.")

# Step 8: Create index on the vector field if it doesn't exist
if not collection.indexes:
    index_params = {
        "metric_type": "IP",  # Inner Product similarity metric
        "index_type": "HNSW",
        "params": {"M": 8, "efConstruction": 64}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    print("Index created on 'embedding' field.")

# Step 9: Load collection into memory to prepare for search queries
collection.load()
print("Collection loaded and ready for queries.")

Connected to Milvus
Loaded existing collection 'appliance_reviews'
Inserted 100 records into Milvus.
Collection loaded and ready for queries.


In [49]:
from sentence_transformers import SentenceTransformer

# Sample query text to search similar reviews
query_text = "This microwave started to make a lot of noise after only a few days"

# Encode the query text using the same embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')
query_embedding = model.encode([query_text]).tolist()

# Load the collection again to be safe
collection = Collection("appliance_reviews")

# Ensure the collection is loaded before searching
collection.load()

# Perform similarity search - get top 5 similar reviews
results = collection.search(
    data=query_embedding,
    anns_field="embedding",
    param={"metric_type": "IP", "params": {"nprobe": 10}},
    limit=5,
    output_fields=["combined_text", "unixReviewTime"]
)

# Print the results with similarity scores
print(f"Search results for query: '{query_text}'\n")
for hit in results[0]:
    print(f"Score: {hit.score:.4f}")
    print(f"Review Text: {hit.entity.get('combined_text')}")
    print(f"Review Time (unix): {hit.entity.get('unixReviewTime')}")
    print("-" * 80)

Search results for query: 'This microwave started to make a lot of noise after only a few days'

Score: 0.2741
Review Text: Five Stars Perfect length to connect my over the range microwave to a wall plug.
Review Time (unix): 1404345600
--------------------------------------------------------------------------------
Score: 0.2741
Review Text: Five Stars Perfect length to connect my over the range microwave to a wall plug.
Review Time (unix): 1404345600
--------------------------------------------------------------------------------
Score: 0.2741
Review Text: Five Stars Perfect length to connect my over the range microwave to a wall plug.
Review Time (unix): 1404345600
--------------------------------------------------------------------------------
Score: 0.2212
Review Text: Great product Not too much to say about this. It works great. We use it in our garage to plug in our stereo when we workout. We use it nearly everyday and haven't had any issues.  It works without fail.
Review Time (

In [51]:
import numpy as np

batch_size = 500  # adjust as needed
total_rows = len(merged_df)
print(f"Total rows to embed and insert: {total_rows}")

for start_idx in range(0, total_rows, batch_size):
    end_idx = min(start_idx + batch_size, total_rows)
    batch_df = merged_df.iloc[start_idx:end_idx].copy()
    
    # Generate embeddings for this batch
    batch_df['embedding'] = batch_df['combined_text'].apply(lambda x: model.encode(x).tolist())
    
    # Prepare insertion data
    embeddings_batch = batch_df['embedding'].tolist()
    texts_batch = batch_df['combined_text'].apply(lambda x: x[:1024]).tolist()
    timestamps_batch = batch_df['unixReviewTime'].tolist()
    
    # Insert batch into Milvus
    collection.insert([embeddings_batch, texts_batch, timestamps_batch])
    print(f"Inserted records from {start_idx} to {end_idx-1}")

print("Finished inserting all records.")

Total rows to embed and insert: 143685
Inserted records from 0 to 499
Inserted records from 500 to 999
Inserted records from 1000 to 1499
Inserted records from 1500 to 1999
Inserted records from 2000 to 2499
Inserted records from 2500 to 2999
Inserted records from 3000 to 3499
Inserted records from 3500 to 3999
Inserted records from 4000 to 4499
Inserted records from 4500 to 4999
Inserted records from 5000 to 5499
Inserted records from 5500 to 5999
Inserted records from 6000 to 6499
Inserted records from 6500 to 6999
Inserted records from 7000 to 7499
Inserted records from 7500 to 7999
Inserted records from 8000 to 8499
Inserted records from 8500 to 8999
Inserted records from 9000 to 9499
Inserted records from 9500 to 9999
Inserted records from 10000 to 10499
Inserted records from 10500 to 10999
Inserted records from 11000 to 11499
Inserted records from 11500 to 11999
Inserted records from 12000 to 12499
Inserted records from 12500 to 12999
Inserted records from 13000 to 13499
Inserted

In [53]:
from sentence_transformers import SentenceTransformer

# Sample query text to search similar reviews
query_text = "This microwave started to make a lot of noise after only a few days"

# Encode the query text using the same embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')
query_embedding = model.encode([query_text]).tolist()

# Load the collection again to be safe
collection = Collection("appliance_reviews")

# Ensure the collection is loaded before searching
collection.load()

# Perform similarity search - get top 5 similar reviews
results = collection.search(
    data=query_embedding,
    anns_field="embedding",
    param={"metric_type": "IP", "params": {"nprobe": 10}},
    limit=5,
    output_fields=["combined_text", "unixReviewTime"]
)

# Print the results with similarity scores
print(f"Search results for query: '{query_text}'\n")
for hit in results[0]:
    print(f"Score: {hit.score:.4f}")
    print(f"Review Text: {hit.entity.get('combined_text')}")
    print(f"Review Time (unix): {hit.entity.get('unixReviewTime')}")
    print("-" * 80)

Search results for query: 'This microwave started to make a lot of noise after only a few days'

Score: 0.6578
Review Text: common microwave problem I'm glad I bought 2. After installing the first one, which worked for a couple  of days another one went bad. It only takes 5 minutes to install, so it's no big deal. My 11 year old microwave is working great again.
Review Time (unix): 1398556800
--------------------------------------------------------------------------------
Score: 0.6386
Review Text: Bought ours at Lowes My microwave has the same problems as the other reviews.  Just got off the phone with the service center.  Whew!  Glad we got the extended warranty.  Will update after service in a couple of days.
Review Time (unix): 1402272000
--------------------------------------------------------------------------------
Score: 0.6255
Review Text: Replacement works great. noise issue resolved. a lot easier than replacing the microwave. It took a little effort to get in to change the m

In [55]:
# Requirement 4 - Create code for semantic similarity search queries using Milvus
# In this code, I embed natural language query texts using the same SentenceTransformer model
# that I used to generate review embeddings. Then, I perform a similarity search on the Milvus
# collection to retrieve the most relevant reviews based on their vector embeddings.
# Finally, I format and display the results including similarity scores, review texts, and timestamps.

from sentence_transformers import SentenceTransformer
from pymilvus import Collection

# Initialize the embedding model once to use for all queries
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load the Milvus collection and ensure it's loaded for search
collection = Collection("appliance_reviews")
collection.load()

def search_similar_reviews(query_text, top_k=5):
    # Embed the input query text to get its vector representation
    query_embedding = model.encode([query_text]).tolist()

    # Perform similarity search in Milvus with the query embedding vector
    results = collection.search(
        data=query_embedding,
        anns_field="embedding",
        param={"metric_type": "IP", "params": {"nprobe": 10}},  # Inner product metric for similarity
        limit=top_k,
        output_fields=["combined_text", "unixReviewTime"]  # Retrieve review text and review timestamp
    )

    # Process and format the search results
    matches = []
    for hit in results[0]:
        matches.append({
            "score": hit.score,
            "review_text": hit.entity.get("combined_text"),
            "review_time": hit.entity.get("unixReviewTime")
        })
    return matches

# Example Group C queries from the requirements to test the search function
queries = [
    "It could so nearly have been a great fridge, but it's the design that brings it down.",
    "This microwave started to make a lot of noise after using it for only few days",
    "Powerful motor and easy to clean blender"
]

# Loop through each query, run the search, and print the top results
for q in queries:
    print(f"Query: {q}")
    results = search_similar_reviews(q)
    for res in results:
        print(f"Score: {res['score']:.4f}, Review: {res['review_text'][:100]}..., Time: {res['review_time']}")
    print("-" * 60)

Query: It could so nearly have been a great fridge, but it's the design that brings it down.
Score: 0.7131, Review: Had this fridge for a couple years now and Love it. I love it. Not a single issue. Everything is gre..., Time: 1397606400
Score: 0.7120, Review: Nice design, poor quality This is a very well designed and well thought out refrigerator. Without qu..., Time: 1350777600
Score: 0.7086, Review: ELEGANT AND PRACTICAL This refrigerator is really slick looking, and that's just the beginning. It's..., Time: 1362268800
Score: 0.7056, Review: Great Fridge! The fridge was exactly what we were looking for and arrived in a timely fashion..., Time: 1280102400
Score: 0.7055, Review: Great fridge I had upgraded from a fridge with the freezer on top and refrigerator on the bottom.  W..., Time: 1389139200
------------------------------------------------------------
Query: This microwave started to make a lot of noise after using it for only few days
Score: 0.6569, Review: common microwave pr

In [57]:
# Requirement 5 - Execute the queries from Requirement 4 and capture the outputs
# This code iterates through all Group C queries, performs similarity search in Milvus,
# and prints the top results for each query for verification and reporting.

from pymilvus import Collection
from sentence_transformers import SentenceTransformer

# Load Milvus collection and embedding model
collection = Collection("appliance_reviews")
collection.load()
model = SentenceTransformer('all-MiniLM-L6-v2')

# Your list of Group C queries (replace or extend with all queries you want to run)
group_c_queries = [
    "It could so nearly have been a great fridge, but it's the design that brings it down.",
    "This microwave started to make a lot of noise after only a few days",
    "Powerful motor and easy to clean blender",
    # Add your other queries here...
]

# Execute each query and print results
for query_text in group_c_queries:
    print(f"Search results for query: '{query_text}'\n")
    
    # Encode the query
    query_embedding = model.encode([query_text]).tolist()
    
    # Perform similarity search, top 5 results
    results = collection.search(
        data=query_embedding,
        anns_field="embedding",
        param={"metric_type": "IP", "params": {"nprobe": 10}},
        limit=5,
        output_fields=["combined_text", "unixReviewTime"]
    )
    
    # Display search results
    for hit in results[0]:
        print(f"Score: {hit.score:.4f}")
        print(f"Review Text: {hit.entity.get('combined_text')}")
        print(f"Review Time (unix): {hit.entity.get('unixReviewTime')}")
        print("-" * 80)
    print("\n" + "="*100 + "\n")

Search results for query: 'It could so nearly have been a great fridge, but it's the design that brings it down.'

Score: 0.7131
Review Text: Had this fridge for a couple years now and Love it. I love it. Not a single issue. Everything is great . Design particularly. Everyone comments about what a nice fridge. I am a proud owner.
Review Time (unix): 1397606400
--------------------------------------------------------------------------------
Score: 0.7120
Review Text: Nice design, poor quality This is a very well designed and well thought out refrigerator. Without question, the nicest fridge I've ever owned.... with one problem. This unit was shipped from the factory with a non-functioning ice maker. After a month of trying to get it fixed, we're shipping it back to Mexico tomorrow. There's really no excuse for a company with a reputation like Electrolux to ship products that don't work. Too bad - we really like the design. But we'll be looking elsewhere now.
Review Time (unix): 13507776